In [1]:
import pandas as pd
import os
import env
import matplotlib.pyplot as plt
import pandas as pd
from pydataset import data
from acquire import get_telco_data
fro

In [4]:
df = get_telco_data()

In [6]:
df = df.loc[:,~df.columns.duplicated()]

In [7]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,Yes,3,Yes,2,90.45,5957.9,No,Two year,DSL,Mailed check
1,0017-DINOC,Male,0,No,No,54,No,No phone service,1,Yes,...,No,3,No,4,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,0019-GFNTW,Female,0,No,No,56,No,No phone service,1,Yes,...,No,3,No,3,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,1,Yes,...,No,3,Yes,4,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,1,No,...,Yes,3,Yes,3,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [8]:
df.columns.to_list()

['customer_id',
 'gender',
 'senior_citizen',
 'partner',
 'dependents',
 'tenure',
 'phone_service',
 'multiple_lines',
 'internet_service_type_id',
 'online_security',
 'online_backup',
 'device_protection',
 'tech_support',
 'streaming_tv',
 'streaming_movies',
 'contract_type_id',
 'paperless_billing',
 'payment_type_id',
 'monthly_charges',
 'total_charges',
 'churn',
 'contract_type',
 'internet_service_type',
 'payment_type']

In [9]:
df_churn = df[df.churn == "Yes"]

In [10]:
df_churn

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
44,0617-AQNWT,Female,0,Yes,No,64,No,No phone service,1,Yes,...,Yes,3,No,1,47.85,3147.5,Yes,Two year,DSL,Electronic check
194,3400-ESFUW,Male,0,Yes,Yes,55,Yes,No,1,Yes,...,No,3,No,3,57.55,3046.4,Yes,Two year,DSL,Bank transfer (automatic)
203,3512-IZIKN,Female,0,Yes,No,70,Yes,Yes,1,Yes,...,No,3,No,4,65.30,4759.75,Yes,Two year,DSL,Credit card (automatic)
253,4355-HBJHH,Male,0,Yes,Yes,67,Yes,Yes,1,Yes,...,Yes,3,Yes,1,79.70,5293.4,Yes,Two year,DSL,Electronic check
326,5458-CQJTA,Male,0,Yes,Yes,65,Yes,No,1,Yes,...,Yes,3,Yes,3,74.80,4820.15,Yes,Two year,DSL,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6992,9025-ZRPVR,Male,0,No,No,9,Yes,No,3,No internet service,...,No internet service,1,Yes,2,18.95,185.6,Yes,Month-to-month,None,Mailed check
6997,9063-ZGTUY,Female,0,Yes,Yes,61,Yes,No,3,No internet service,...,No internet service,1,Yes,2,19.40,1182.55,Yes,Month-to-month,None,Mailed check
7005,9174-FKWZE,Female,1,No,No,1,Yes,No,3,No internet service,...,No internet service,1,Yes,1,19.45,19.45,Yes,Month-to-month,None,Electronic check
7012,9318-NKNFC,Male,0,No,No,1,Yes,No,3,No internet service,...,No internet service,1,Yes,2,18.85,18.85,Yes,Month-to-month,None,Mailed check
